# 2021 빅데이터 과목 
# 시흥시 연계 프로젝트 2차
## 시흥시 버스 포화도 분석 및 포화도에 따른 탄력배차 제안

#
#

### 참여자 : 이영채 , 김동혁


## 경기도 정류소 좌표를 기준으로 시흥시를 경유하는 모든 노선을 파악

### 정류소 좌표데이터 : 경기도 정류소 좌표.csv

In [1]:
from urllib.request import Request,urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup
import xmltodict
import json
import pandas as pd

serviceKey='A1p0vUiD/eSU99PG7kAIIwmJXcA9VJJuPChc5gRjevr4XDnF852eQpbgiD5mUcfArWRi9g4vt8S+heg2sFsWaA=='

In [2]:
#정류장 id를 이용하여 데이터 뽑기
def stationInfo(stationId):
    url = 'http://apis.data.go.kr/6410000/busstationservice/getBusStationViaRouteList'
    queryParams = '?' + urlencode({ quote_plus('serviceKey') : serviceKey, quote_plus('stationId') : stationId })

    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = xmltodict.parse(urlopen(request).read().decode('utf8'))
    return response_body

In [3]:
def readStationData():
    data = pd.read_csv("경기도정류소좌표/경기도 정류소 좌표.csv")
    return data['STATION_ID']

In [4]:
df = pd.DataFrame(index=range(0,0),columns=['routeName','routeId','routeType','routeTypeName'])
stationList=readStationData()
for stationid in stationList:
    info = stationInfo(stationid)
    print(stationid)
    try:
        for bus in info['response']['msgBody']['busRouteList']:
            routeName = bus['routeName']
            routeId = bus['routeId']
            routeType = bus['routeTypeCd']
            routeTypeName = bus['routeTypeName']
            df = df.append({'routeName':routeName,'routeId':routeId,'routeType':routeType,'routeTypeName':routeTypeName}, ignore_index=True)
    except TypeError:
        routeName = info['response']['msgBody']['busRouteList']['routeName']
        routeId = info['response']['msgBody']['busRouteList']['routeId']
        routeType = info['response']['msgBody']['busRouteList']['routeTypeCd']
        routeTypeName = info['response']['msgBody']['busRouteList']['routeTypeName']
        df = df.append({'routeName':routeName,'routeId':routeId,'routeType':routeType,'routeTypeName':routeTypeName}, ignore_index=True)
    except:
        continue

224000001
224000002
224000003
224000004
224000005
224000006
224000007
224000008
224000009
224000010
224000011
224000012
224000013
224000014
224000015
224000016
224000017
224000018
224000019
224000020
224000021
224000022
224000023
224000024
224000025
224000026
224000027
224000028
224000029
224000030
224000031
224000032
224000033
224000034
224000035
224000036
224000037
224000038
224000039
224000040
224000041
224000042
224000043
224000044
224000045
224000046
224000047
224000048
224000049
224000050
224000051
224000052
224000054
224000055
224000056
224000057
224000058
224000059
224000060
224000061
224000062
224000063
224000064
224000065
224000066
224000067
224000068
224000069
224000070
224000071
224000072
224000074
224000075
224000076
224000077
224000078
224000080
224000081
224000082
224000083
224000084
224000085
224000086
224000087
224000088
224000089
224000090
224000091
224000092
224000093
224000094
224000095
224000096
224000097
224000098
224000099
224000100
224000101
224000102
224000103


In [5]:
#중복제거
df=df.drop_duplicates()
df.to_excel("시흥시 모든 버스 리스트.xlsx", encoding='euc-kr',index = False)

In [6]:
#시흥시를 경유하는 모든 노선의 DataFrame
df

,routeName,routeId,routeType,routeTypeName
0,11,213000008,13,일반형시내버스
1,31-7,216000019,13,일반형시내버스
2,32,217000009,13,일반형시내버스
3,35,217000010,13,일반형시내버스
4,350,217000012,13,일반형시내버스
...,...,...,...,...
2689,97,216000040,13,일반형시내버스
2889,G8808,210000108,11,직행좌석형시내버스
2890,8106,234001237,16,경기외곽순환버스
2891,8906,235000103,16,경기외곽순환버스


## 시흥시를 경유하는 모든 노선에 대한 차량 배차, 첫차, 막차 시간 데이터

### : 경기도 내 버스 노선별 배차시간_첫차_막차.xlsx  


##### 해당파일의 경우 경기도의 모든 버스노선 배차시간파일에서 추출함.

In [7]:
import pandas as pd

import numpy as np

## 평균배차시간을 구하는 함수를 정의

In [8]:
def weekday_time(busdf,bus_number):  #이는 평균 배차시간을 뽑는 함수

    #bus_num = '1-A'
    dl = busdf[busdf['노선번호'] == bus_number]

    print("주중배차간격 :{} ".format(str(dl[['주중배차간격']]).replace('\n','').split(' ')[-1]))
    batch = str(dl[['주중배차간격']]).replace('\n','').split(' ')[-1].split('~')

    sum = 0
    for nn in batch:
        nn = nn.replace('분','')
        sum += int(nn)
    print('평균 배차시간 : {}'.format(sum/len(batch)))

    return sum/len(batch)

## 사전파일 정제

In [9]:
df3 = pd.read_excel('버스/시흥시 경유노선 21년 4월.xlsx')
df3 = df3.rename(columns=df3.iloc[2])
df3 = df3.drop([0,1,2])
df3 = df3.dropna()
df3.drop(columns = ['단일승차','단일하차','단일전체','환승승차','환승하차','환승전체'],inplace = True)
df3


,지역,노선,시·종점,월,일시,시간,승차,하차,전체
3,시흥시,12,삼미시장-김포공항국내선(9번홈),4,평일,01시~02시,1,2,3
4,시흥시,12,삼미시장-김포공항국내선(9번홈),4,평일,04시~05시,32,1,33
5,시흥시,12,삼미시장-김포공항국내선(9번홈),4,평일,05시~06시,66,76,142
6,시흥시,12,삼미시장-김포공항국내선(9번홈),4,평일,06시~07시,196,146,342
7,시흥시,12,삼미시장-김포공항국내선(9번홈),4,평일,07시~08시,515,445,960
...,...,...,...,...,...,...,...,...,...
1238,광명시,39,매화초등학교-개봉역,4,평일,20시~21시,97,129,226
1239,광명시,39,매화초등학교-개봉역,4,평일,21시~22시,77,93,170
1240,광명시,39,매화초등학교-개봉역,4,평일,22시~23시,73,67,140
1241,광명시,39,매화초등학교-개봉역,4,평일,23시~24시,25,44,69


## 노선별 배차시간 정보 호출

In [10]:
# 노선별 배차시간 불러오기
bus_time_ = pd.read_excel('경기도 내 버스 노선별 배차시간_첫차_막차.xlsx')
bus_time = bus_time_.astype({'노선운행시간' : int}).drop_duplicates(['노선번호'])

## 포화도 분석

In [11]:


#포화도 결과 리스트
result = []

#배차시간 리스트
bTime = []

#root_time 노선운행시간
root_time_List = []

for num in range(len(df3)):
    print("=============")
    print("분석노선 : {}".format(df3['노선'].iloc[num]))
    print("시간당 승차승객 : {}".format(df3['승차'].iloc[num]))
    
    bTime.append(weekday_time(bus_time, df3['노선'].iloc[num]))
    
    #노선운행시간 계산
    root_time = int(bus_time[bus_time['노선번호'] == df3['노선'].iloc[num]] ['노선운행시간'])
    
    #노선운행시간리스트
    root_time_List.append(root_time)
    
    #노선주행이 60보다 클경우
    if root_time > 60:
        #동시간대 돌아다니는 차량대수 = 편도노선운행시간 * 2 / 평균배차시간
        bus_onPath = ( 2 * bus_time[bus_time['노선번호'] == df3['노선'].iloc[num]] ['노선운행시간'] ) / weekday_time(bus_time, df3['노선'].iloc[num])
        print("해당 시간내 주행중인 버스 대수 : {}".format(round(float(bus_onPath),2)))
    
    else:   #노선주행시간이 60분 미만인 노선 대상
        
        #동시간대 돌아다니는 차량대수 = 60분 * 2 / 평균배차시간
        bus_onPath = ( 2 * 60 ) / weekday_time(bus_time, df3['노선'].iloc[num])
        print("해당 시간내 주행중인 버스 대수 : {}".format(round(float(bus_onPath),2)))
        

    # 버스 한대당 탄 승객수 = 승차인원수 / 동시간대 돌아다니는 차량 대수
    one_BusPass = int(df3['승차'].iloc[num]) / float(bus_onPath)

    #포화도 = (버스 한대당 탄 승객수 / 버스 적정 탑승인원) * 100
    result.append(round(one_BusPass / 40 , 2) * 100)


print(result)

격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 1212
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 1371
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 898
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 703
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 580
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 567
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 246
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 : 1(광명)
시간당 승차승객 : 50
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
주중배차간격 :7분~10분 
평균 배차시간 : 8.5
해당 시간내 주행중인 버스 대수 : 25.41
분석노선 :

## 분석 정보 데이터프레임에 추가 및 포화도 순으로 데이터 프레임 정렬

In [12]:

df3['주중 평균배차시간'] = bTime
df3['노선운행시간'] = root_time_List
df3['시간별 포화도'] = result


df3.sort_values('시간별 포화도',ascending = False,inplace = True)

## 분석결과 xlsx 형태로 저장

In [13]:
def toCsv(name, df):
    tmp = df.copy()
    for i,row in tmp.iterrows():
        #저장후 노선 번호 이상문제 ' '추가로 해결
        tmp.at[i,'노선'] = ' '+tmp.at[i,'노선']
        
    tmp.to_csv(name, encoding ='cp949', index = None)

df3.to_excel('포화도결과파일/포화도 계산 21년 4월.xlsx')
toCsv('포화도결과파일/포화도 계산 21년 4월.csv',df3)
# df3.to_csv('포화도결과파일/포화도 계산 21년 4월.csv',encoding='euc-kr')

In [14]:
df3

,지역,노선,시·종점,월,일시,시간,승차,하차,전체,주중 평균배차시간,노선운행시간,시간별 포화도
81,시흥시,20-1,정왕역환승센터-부곡중학교,4,평일,07시~08시,761,706,1467,15.0,78,183.0
132,시흥시,23,월곶역-본오동차고지,4,평일,18시~19시,1072,"1,005",2077,12.5,99,169.0
1225,광명시,39,매화초등학교-개봉역,4,평일,07시~08시,386,298,684,20.5,45,165.0
1166,광명시,11,광명돔경륜장.광남문-안양역,4,평일,08시~09시,262,274,536,32.5,65,164.0
8,시흥시,12,삼미시장-김포공항국내선(9번홈),4,평일,08시~09시,585,630,1215,15.0,67,164.0
...,...,...,...,...,...,...,...,...,...,...,...,...
963,안산시,55,본오동종점-오이도차고지,4,평일,00시~01시,0,3,3,9.0,93,0.0
1063,안산시,98,오이도차고지-수변공원휴게소,4,평일,00시~01시,0,1,1,27.5,104,0.0
1103,부천시,23-2,소사공영차고지-라일락마을어린이공원,4,평일,00시~01시,2,9,11,8.5,69,0.0
1141,부천시,G8808,송내역남부(광역버스)-KTX광명역5번출구,4,평일,22시~23시,0,3,3,30.0,31,0.0


## 1~5월 데이터 뽑기 위해 구동하는 코드 : 함수와 분석대상 파일 존재



### 이미 구해놓았기에 다시 돌릴 필요 X

In [15]:

def files_processing(df33 , string):

    #df3 = pd.read_excel('버스/시흥시 경유노선 21년 4월.xlsx')
    df33 = df33.rename(columns=df33.iloc[2])
    df33 = df33.drop([0,1,2])
    df33 = df33.dropna()
    df33.drop(columns = ['단일승차','단일하차','단일전체','환승승차','환승하차','환승전체'],inplace = True)

    # 노선별 배차시간 불러오기
    bus_time_ = pd.read_excel('경기도 내 버스 노선별 배차시간_첫차_막차.xlsx')
    bus_time = bus_time_.astype({'노선운행시간' : int}).drop_duplicates(['노선번호'])




    #포화도 결과 리스트
    result = []

    #배차시간 리스트
    bTime = []

    #root_time 노선운행시간
    root_time_List = []

    for num in range(len(df33)):
        # print("=============")
        # print("분석노선 : {}".format(df3['노선'].iloc[num]))
        # print("시간당 승차승객 : {}".format(df3['승차'].iloc[num]))
        
        bTime.append(weekday_time(bus_time, df33['노선'].iloc[num]))
        
        #노선운행시간 계산
        root_time = int(bus_time[bus_time['노선번호'] == df33['노선'].iloc[num]] ['노선운행시간'])
        
        #노선운행시간리스트
        root_time_List.append(root_time)
        
        #노선주행이 60보다 클경우
        if root_time > 60:
            #동시간대 돌아다니는 차량대수 = 편도노선운행시간 * 2 / 평균배차시간
            bus_onPath = ( 2 * bus_time[bus_time['노선번호'] == df33['노선'].iloc[num]] ['노선운행시간'] ) / weekday_time(bus_time, df33['노선'].iloc[num])
            # print("해당 시간내 주행중인 버스 대수 : {}".format(round(float(bus_onPath),2)))
        
        else:   #노선주행시간이 60분 미만인 노선 대상
            
            #동시간대 돌아다니는 차량대수 = 60분 * 2 / 평균배차시간
            bus_onPath = ( 2 * 60 ) / weekday_time(bus_time, df33['노선'].iloc[num])
            # print("해당 시간내 주행중인 버스 대수 : {}".format(round(float(bus_onPath),2)))
            

        # 버스 한대당 탄 승객수 = 승차인원수 / 동시간대 돌아다니는 차량 대수
        one_BusPass = int(df33['승차'].iloc[num]) / float(bus_onPath)

        #포화도 = (버스 한대당 탄 승객수 / 버스 적정 탑승인원) * 100
        result.append(round(one_BusPass / 40 , 2) * 100)



    df33['주중 평균배차시간'] = bTime
    df33['노선운행시간'] = root_time_List
    df33['시간별 포화도'] = result


    df33.sort_values('시간별 포화도',ascending = False,inplace = True)

    df33.to_excel('filename.xlsx'.replace('filename',string))

    toCsv('filename.csv'.replace('filename',string),df33)
    #df33.to_csv()

In [16]:
df = pd.read_excel('버스/시흥시 경유노선 21년 1월.xlsx')
files_processing(df , '포화도결과파일/시흥시 경유노선 21년 1월 포화도 산출파일')

df = pd.read_excel('버스/시흥시 경유노선 21년 2월.xlsx')
files_processing(df , '포화도결과파일/시흥시 경유노선 21년 2월 포화도 산출파일')

df = pd.read_excel('버스/시흥시 경유노선 21년 3월.xlsx')
files_processing(df , '포화도결과파일/시흥시 경유노선 21년 3월 포화도 산출파일')

df = pd.read_excel('버스/시흥시 경유노선 21년 4월.xlsx')
files_processing(df , '포화도결과파일/시흥시 경유노선 21년 4월 포화도 산출파일')

df = pd.read_excel('버스/시흥시 경유노선 21년 5월.xlsx')
files_processing(df , '포화도결과파일/시흥시 경유노선 21년 5월 포화도 산출파일')


평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간 : 22.5
주중배차간격 :15분~30분 
평균 배차시간

## 분석된 자료에 따라 증차 / 감차 계산

In [17]:
import pandas as pd
import numpy as np
#pd.options.display.float_format = '{:.2f}'.format

#적정 탑승인원 40을 기준으로 포화도 100
std = 40
std90 = std*0.9

## 분석대상 파일을 읽고 시간당 총 운행중인 버스 대수를 구하는 함수 구현

In [18]:
def readSaturation(name):
    # df=pd.read_excel(name)
    df=pd.read_csv(name,encoding='cp949')
    #df=df.drop(['Unnamed: 0'],axis=1)
    df=df.drop(['지역'],axis=1)
    df=df.drop(['일시'],axis=1)
    df=df.drop(['월'],axis=1)
    df=df.drop(['하차'],axis=1)
    df=df.drop(['전체'],axis=1)

    for i,row in df.iterrows():    

        #왕복 운행 대수
        #편도 운행시간 60분 이상 노선들
        if df.at[i,'노선운행시간']>=60:
            df.at[i,'왕복운행대수']=round((df.at[i,'노선운행시간']/df.at[i,'주중 평균배차시간']*2),2)
        #편도 운행시간 60분 이하 노선들
        else:
            df.at[i,'왕복운행대수']=round((60/df.at[i,'주중 평균배차시간']*2),2)
    return df

## 동적배차시간을 구하는 함수
### 현실적으로 증차 대수는 기존 운행되는 버스대수대비 130%로 제한
### 추후 증차할 차량수를 커버할수 있는 동일 노선 감차를 계산하는데에도 쓰임

In [19]:
#동적 배차시간 만들기
def dynamicTime(df):
    dfU=df[df['시간별 포화도']>=100]
    dfU=dfU.reset_index(drop=True) 

    #포화도를 90퍼센트까지 내리는 것으로 목표
    for i,row in dfU.iterrows():    

        #단순히 증차한 운행대수 (승차 인원/포화도90기준차량승객수)
        dfU.at[i,'증차운행대수']= round((dfU.at[i,'승차']/std90),2)

        #증차를 기존 운행대수 130%로 제한한 현실적인 운행대수
        if dfU.at[i,'증차운행대수']>= dfU.at[i,'왕복운행대수']*1.3:
            dfU.at[i,'증차운행대수']=round((dfU.at[i,'왕복운행대수']*1.3),2)

        #증차 대수
        dfU.at[i,'증차대수'] = dfU.at[i,'증차운행대수']-dfU.at[i,'왕복운행대수']

        #증차 이후 배차시간
        if dfU.at[i,'노선운행시간']>=60:
            dfU.at[i, '증차후배차시간'] = round((dfU.at[i,'노선운행시간']/dfU.at[i,'증차운행대수']*2),2)
        #편도 운행시간 60분 이하 노선들
        else:
            dfU.at[i, '증차후배차시간'] = round((60/dfU.at[i,'증차운행대수']*2),2)


        #증차 이후 예상 포화도
        dfU.at[i,'증차후예상포화도'] = round((dfU.at[i,'승차']/(dfU.at[i,'증차운행대수']*std)*100),2)

        #감소한 포화도
        dfU.at[i, '포화도감소율'] = dfU.at[i,'시간별 포화도']-dfU.at[i,'증차후예상포화도']
    return dfU

## 감차 차량수 구하는 함수
### 증차 차량수를 채우기 위해 특정시간대에서 감차할 차량수를 구함
### 기존데이터와 증차차량데이터를 통해 구함

In [44]:
#증차 대체 감차 운행시간 찾기
#기존 데이터와 동적 배차시간 데이터 필요
def compare(df,dfU):
    dfL=dfU.copy()
    dfL=dfL.drop(['승차'],axis=1)
    dfL=dfL.drop(['시·종점'],axis=1)
    dfL=dfL.drop(['포화도감소율'],axis=1)
    
    for i,row in dfL.iterrows():
        busNum=dfL.at[i,'노선']
        
        #포화도 높은 순과 낮은 순의 데이터를 정렬하여 1대1로 증차와 감차
        #포화도 100이 넘어가는 중복되는 노선의 순서, 순서는 0에 가까울수록 포화도가 높음
        #포화도 순서 찾기
        count=0
        for j,jrow in dfL.iterrows():
            if i==j: break
            if dfL.at[i,'노선']==dfL.at[j,'노선']: count=count+1
        #노선의 감차 시간대 데이터 찾아서 리스트에 추가
        #가장 승차수 적은 시간대 찾기(7시 ~ 22시 사이)
        #이외의 시간들은 종료 시점이라 의미 없을 가능성 多
        leastlist=[]
        for a in range(12):
            #시간별 포화도의 오름차순으로 정렬해서 포화도 낮은순으로 리스트에 추가
            l=df[df['노선']==busNum].sort_values(by=['시간별 포화도'],axis=0,ascending=True).iloc[a]
            if l['시간'] not in ['22시~23시','23시~24시','00시~01시','01시~02시','02시~03시','03시~04시','04시~05시','05시~06시','06시~07시']:
                leastlist.append(l)
        
        #print(count)
        #감차할 시간대 데이터
        #if count > 6 : count == 6
        least=leastlist[count]
        
        
        #노선 감차 시간대 데이터 dfL에 추가
        dfL.at[i,'감차대상시간']=least['시간']
        dfL.at[i,'감차대상포화도']=least['시간별 포화도']
        dfL.at[i,'감차운행대수']=dfL.at[i,'왕복운행대수']-dfL.at[i,'증차대수']

        #감차후배차시간 계산
        if dfL.at[i,'노선운행시간']>=60:
            dfL.at[i,'감차후배차시간']=round((dfL.at[i,'노선운행시간']/dfL.at[i,'감차운행대수']*2),2)
        #편도 운행시간 60분 이하 노선들
        else:
            dfL.at[i,'감차후배차시간'] = round((60/dfL.at[i,'감차운행대수']*2),2)

        #감차 이후 예상 포화도
        dfL.at[i,'감차후예상포화도'] = round((least['승차']/(dfL.at[i,'감차운행대수']*std)*100),2)
        
    #노선 번호로 Sort
    dfL=dfL.sort_values(by=['노선'],axis=0,ascending = True)
    return dfL

In [21]:
def toCsv(name, df):
    tmp = df.copy()
    for i,row in tmp.iterrows():
        #저장후 노선 번호 이상문제 ' '추가로 해결
        tmp.at[i,'노선'] = ' '+tmp.at[i,'노선']
        
    tmp.to_csv(name, encoding ='cp949', index = None)

## 21년 4월 데이터를 기준으로 코드실행

In [45]:
#df=readSaturation("시흥시 경유노선 21년 4월 포화도 산출파일.xlsx")
df=readSaturation("포화도결과파일/시흥시 경유노선 21년 4월 포화도 산출파일.csv")
df

,노선,시·종점,시간,승차,주중 평균배차시간,노선운행시간,시간별 포화도,왕복운행대수
0,20-1,정왕역환승센터-부곡중학교,07시~08시,761,15.0,78,183.0,10.40
1,23,월곶역-본오동차고지,18시~19시,1072,12.5,99,169.0,15.84
2,39,매화초등학교-개봉역,07시~08시,386,20.5,45,165.0,5.85
3,11,광명돔경륜장.광남문-안양역,08시~09시,262,32.5,65,164.0,4.00
4,12,삼미시장-김포공항국내선(9번홈),08시~09시,585,15.0,67,164.0,8.93
...,...,...,...,...,...,...,...,...
1235,55,본오동종점-오이도차고지,00시~01시,0,9.0,93,0.0,20.67
1236,98,오이도차고지-수변공원휴게소,00시~01시,0,27.5,104,0.0,7.56
1237,23-2,소사공영차고지-라일락마을어린이공원,00시~01시,2,8.5,69,0.0,16.24
1238,G8808,송내역남부(광역버스)-KTX광명역5번출구,22시~23시,0,30.0,31,0.0,4.00


In [46]:
dfU=dynamicTime(df)
toCsv("탄력적배차결과파일/4월 동적 배차시간.csv",dfU)
dfU

,노선,시·종점,시간,승차,주중 평균배차시간,노선운행시간,시간별 포화도,왕복운행대수,증차운행대수,증차대수,증차후배차시간,증차후예상포화도,포화도감소율
0,20-1,정왕역환승센터-부곡중학교,07시~08시,761,15.0,78,183.0,10.40,13.52,3.12,11.54,140.72,42.28
1,23,월곶역-본오동차고지,18시~19시,1072,12.5,99,169.0,15.84,20.59,4.75,9.62,130.16,38.84
2,39,매화초등학교-개봉역,07시~08시,386,20.5,45,165.0,5.85,7.60,1.75,15.79,126.97,38.03
3,11,광명돔경륜장.광남문-안양역,08시~09시,262,32.5,65,164.0,4.00,5.20,1.20,25.00,125.96,38.04
4,12,삼미시장-김포공항국내선(9번홈),08시~09시,585,15.0,67,164.0,8.93,11.61,2.68,11.54,125.97,38.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,20,경원여객-시흥시청후문.종합일자리센터,08시~09시,69,70.0,40,101.0,1.71,1.92,0.21,62.50,89.84,11.16
66,81,목감호수품애.중흥S클래스-안양역,08시~09시,368,13.0,39,100.0,9.23,10.22,0.99,11.74,90.02,9.98
67,55,본오동종점-오이도차고지,18시~19시,824,9.0,93,100.0,20.67,22.89,2.22,8.13,90.00,10.00
68,32,경원여객안양차고지-풍림2단지.소래포구입구,18시~19시,275,25.0,86,100.0,6.88,7.64,0.76,22.51,89.99,10.01


In [47]:
dfL=compare(df,dfU)
toCsv("탄력적배차결과파일/4월 증차, 감차 대상 비교 분석.csv",dfL)
dfL

,노선,시간,주중 평균배차시간,노선운행시간,시간별 포화도,왕복운행대수,증차운행대수,증차대수,증차후배차시간,증차후예상포화도,감차대상시간,감차대상포화도,감차운행대수,감차후배차시간,감차후예상포화도
25,1(광명),08시~09시,8.5,108,125.0,25.41,33.03,7.62,6.54,96.20,11시~12시,61.0,17.79,12.14,86.85
18,1(광명),18시~19시,8.5,108,135.0,25.41,33.03,7.62,6.54,103.77,21시~22시,57.0,17.79,12.14,81.51
30,1(광명),17시~18시,8.5,108,119.0,25.41,33.03,7.62,6.54,91.73,12시~13시,68.0,17.79,12.14,96.96
59,1(광명),16시~17시,8.5,108,102.0,25.41,28.78,3.37,7.51,89.99,13시~14시,69.0,22.04,9.80,79.29
7,1(광명),07시~08시,8.5,108,151.0,25.41,33.03,7.62,6.54,116.03,10시~11시,56.0,17.79,12.14,79.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,76,07시~08시,90.0,86,126.0,1.91,2.48,0.57,69.35,96.77,16시~17시,25.0,1.34,128.36,35.45
33,81,07시~08시,13.0,39,118.0,9.23,12.00,2.77,10.00,91.04,11시~12시,42.0,6.46,18.58,59.60
66,81,08시~09시,13.0,39,100.0,9.23,10.22,0.99,11.74,90.02,14시~15시,48.0,8.24,14.56,54.31
54,98,07시~08시,27.5,104,106.0,7.56,8.89,1.33,23.40,89.99,11시~12시,33.0,6.23,33.39,40.13


In [48]:

df=readSaturation("포화도결과파일/시흥시 경유노선 21년 2월 포화도 산출파일.csv")
dfU=dynamicTime(df)
toCsv("탄력적배차결과파일/2월 동적 배차시간.csv",dfU)
dfL=compare(df,dfU)
toCsv("탄력적배차결과파일/2월 증차, 감차 대상 비교 분석.csv",dfL)

df=readSaturation("포화도결과파일/시흥시 경유노선 21년 3월 포화도 산출파일.csv")
dfU=dynamicTime(df)
toCsv("탄력적배차결과파일/3월 동적 배차시간.csv",dfU)
dfL=compare(df,dfU)
toCsv("탄력적배차결과파일/3월 증차, 감차 대상 비교 분석.csv",dfL)

df=readSaturation("포화도결과파일/시흥시 경유노선 21년 5월 포화도 산출파일.csv")
dfU=dynamicTime(df)
toCsv("탄력적배차결과파일/5월 동적 배차시간.csv",dfU)
dfL=compare(df,dfU)
toCsv("탄력적배차결과파일/5월 증차, 감차 대상 비교 분석.csv",dfL)

IndexError: list index out of range

In [43]:
df111=readSaturation("포화도결과파일/시흥시 경유노선 21년 1월 포화도 산출파일.csv")
df111
dfU111=dynamicTime(df111)
toCsv("탄력적배차결과파일/1월 동적 배차시간.csv",dfU111)
dfU111
dfL111=compare(df111,dfU111)
toCsv("탄력적배차결과파일/1월 증차, 감차 대상 비교 분석.csv",dfL111)
dfL111

IndexError: single positional indexer is out-of-bounds